In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, SimpleRNN, LSTM, GRU, Dense

In [2]:
num_words= 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)

In [3]:
maxlen = 100
x_train = pad_sequences(x_train, maxlen=maxlen, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=maxlen, padding='post', truncating='post')

In [4]:
version_fn = getattr(tf.keras, "version", None)
if version_fn and version_fn().startswith("3."):
    import tensorflow.keras as keras
else:
    keras = tf.keras

In [5]:
from tensorflow.keras.layers import Input, Dropout
from tensorflow.keras.models import Model
embedding =100
#encoder_input = Input(shape=(maxlen,))
#model = Sequential([
 #   Embedding(input_dim=num_words, output_dim=embedding, input_length=maxlen),
 #   Bidirectional(LSTM(5)),
 #   Dense(1, activation='sigmoid')
#])
input_layer = Input(shape=(maxlen,))
x = Embedding(input_dim=num_words, output_dim=embedding)(input_layer)
x = Bidirectional(LSTM(5, return_sequences=False))(x)
x = Dropout(0.2)(x)
output_layer = Dense(1, activation='sigmoid')(x)
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train,batch_size=num_words, epochs=1)
model.build(input_shape=(1,maxlen))


3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 953ms/step - accuracy: 0.5197 - loss: 0.6926


In [6]:
dummy_input = tf.zeros((1, maxlen))

# Call the model with dummy input to initialize it
output = model(dummy_input)

# Print the model summary to verify
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 100)       │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 10)             │         4,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,012,755 (11.49 MB)

 Trainable params: 1,004,251 (3.83 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,008,504 (7.66 MB)

In [7]:
train_loss, train_acc = model.evaluate(x_test, y_test, verbose=2)

782/782 - 8s - 11ms/step - accuracy: 0.5696 - loss: 0.6900


In [8]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [9]:
log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [10]:
log_dir

'logs/fit20240726-102054'

In [11]:
model_callbacks = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [12]:
import mlflow
earLy_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [13]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, callbacks=[model_callbacks, earLy_stop]
)

Epoch 1/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 44s 55ms/step - accuracy: 0.7002 - loss: 0.5757 - val_accuracy: 0.7942 - val_loss: 0.4409
Epoch 2/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 50s 64ms/step - accuracy: 0.8688 - loss: 0.3303 - val_accuracy: 0.8095 - val_loss: 0.4321
Epoch 3/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 51s 65ms/step - accuracy: 0.9140 - loss: 0.2336 - val_accuracy: 0.8048 - val_loss: 0.4740
Epoch 4/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 23s 29ms/step - accuracy: 0.9379 - loss: 0.1727 - val_accuracy: 0.7956 - val_loss: 0.5320
Epoch 5/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 22s 28ms/step - accuracy: 0.9581 - loss: 0.1273 - val_accuracy: 0.7978 - val_loss: 0.6061
Epoch 6/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - accuracy: 0.9702 - loss: 0.0966 - val_accuracy: 0.7891 - val_loss: 0.6978
Epoch 7/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 27ms/step - accuracy: 0.9818 - loss: 0.0637 - val_accuracy: 0.7867 - val_loss: 0.7688
Epoch 8/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 26ms/step - accuracy: 0.9865 - loss: 0

In [14]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 100)       │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 10)             │         4,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,012,755 (11.49 MB)

 Trainable params: 1,004,251 (3.83 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,008,504 (7.66 MB)

In [15]:
from mlflow.models.signature import infer_signature
model_sign = infer_signature(x_test, model.predict(x_test))

782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step


In [16]:
exp_timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
exp_name= "IMDB_bidirection_lstm_time:" + exp_timestamp
#exp_name = "IMDB_bidirection_lstm"
run_timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
run_name = "IMDB_bidirection_lstm_run:" + run_timestamp

In [17]:
def tf_experiment_tracking(exp_name, run_name, train_loss, train_acc, model_sign):
    mlflow.set_experiment(exp_name)
    with mlflow.start_run(run_name=run_name):
        mlflow.log_param("batch_size", num_words )
        mlflow.log_param("learning_rate", 0.001)
        mlflow.log_param("epochs", 100)
        mlflow.log_metric("train_loss", train_loss)
        mlflow.log_metric("train_accuracy", train_acc)
        mlflow.tensorflow.log_model(model, "IMDB", signature=model_sign)
    mlflow.end_run()

In [18]:
tf_experiment_tracking(exp_name, run_name, train_loss, train_acc, model_sign)

2024/07/26 10:26:32 INFO mlflow.tracking.fluent: Experiment with name 'IMDB_bidirection_lstm_time:20240726-102632' does not exist. Creating a new experiment.


In [19]:
model.export(filepath='saved_model', format='tf')
model.save('models/BI_LSTM_model.h5')
model.save_weights('models/LSTM.weights.h5')

INFO:tensorflow:Assets written to: saved_model\assets


INFO:tensorflow:Assets written to: saved_model\assets


Saved artifact at 'saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2061770325264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2061771801040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2061771803536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2061771802768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2061771802192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2061771803344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2061771804304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2061771802576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2061771805264: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [21]:
import bentoml

bento_model = bentoml.tensorflow.save_model("BI_LSTM_model", model)

INFO:tensorflow:Assets written to: C:\Users\HP\AppData\Local\Temp\tmpzuvxmmu3bentoml_model_bi_lstm_model\assets


INFO:tensorflow:Assets written to: C:\Users\HP\AppData\Local\Temp\tmpzuvxmmu3bentoml_model_bi_lstm_model\assets
